In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score


# ---------------------------------------------------------
# Sigmoid Function
# ---------------------------------------------------------
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


# ---------------------------------------------------------
# Logistic Regression Model (From Scratch)
# ---------------------------------------------------------
class LogisticRegressionScratch:
    def __init__(self, lr=0.01, max_iters=5000):
        self.lr = lr
        self.max_iters = max_iters

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.max_iters):
            linear_output = np.dot(X, self.weights) + self.bias
            predictions = sigmoid(linear_output)

            # Gradients
            dw = (1 / n_samples) * np.dot(X.T, (predictions - y))
            db = (1 / n_samples) * np.sum(predictions - y)

            # Update parameters
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        probs = sigmoid(linear_output)
        return np.where(probs >= 0.5, 1, 0)


# ---------------------------------------------------------
# 1. Generate Synthetic Binary Classification Dataset
# ---------------------------------------------------------
X, y = make_classification(
    n_samples=500,
    n_features=2,
    n_redundant=0,
    n_clusters_per_class=1,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# ---------------------------------------------------------
# 2. Train Model
# ---------------------------------------------------------
model = LogisticRegressionScratch(lr=0.1, max_iters=3000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# ---------------------------------------------------------
# 3. Evaluation
# ---------------------------------------------------------
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Final Weights:", model.weights)
print("Final Bias:", model.bias)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# ---------------------------------------------------------
# 4. Decision Boundary Visualization
# ---------------------------------------------------------
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

xx, yy = np.meshgrid(
    np.linspace(x_min, x_max, 200),
    np.linspace(y_min, y_max, 200)
)

grid = np.c_[xx.ravel(), yy.ravel()]
Z = model.predict(grid)
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X[:, 0], X[:, 1], c=y, s=20)
plt.title("Decision Boundary (Logistic Regression from Scratch)")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.savefig("decision_boundary.png")  # Required deliverable
plt.close()

TypeError: CustomKNN() takes no arguments